# Pivoting

Pandas strives to be very expressive in terms of one-liners, taking advantage of some decades of accumlated spreadsheet research, along with SQL, an old standby for data manipulation.

Pivoting is a reshaping operation involving swapping columns for rows.  I find it difficult to master when the indexes are hierarchical to begin with.  My tendency is to reduce such tables to a simpler form before attempting any pivoting.

Pivot is a method of all DataFrames, as is pivot_table.  You may also invoke them from the pandas module level, e.g. ```pd.pivot_table(table_name, ...)```.

"Wouldn't you rather use SQL?"  Lets remember that SQL was envisioned as a broadly accessible solution to the challenge of storage and retrieval.  NoSQL is just as much a solution and the two together dominate the warehouse of persistent data.

Pandas is more likely middleware between the warehouse and some end user, a client for the summary visualizations that Jupyter Notebooks are so good at displaying.

However, just as SQLite text databases may be useful for sharing structured information, almost at the same level as JSON and XML, so may pandas gradually make inroads as a common file format.  Indeed, this is the current vision of Wess McKinney, to standardize a common layer for data exchange, based on this dataframe paradigm.  R has it as well.

In [1]:
import pandas as pd

I did use Google to get some real TV stations per each city, but the ratings column is meaningless. I'm just wanting to show how pivoting works.

In [2]:
table = pd.DataFrame(
   {"city": ["Portland", "Portland", "Sacramento", "Sacramento", "Austin", "Austin", "Austin"],
    "TV": ["KOIN", "KABC", "KCRA", "KXTV", "KVUE", "KXAN", "KTBC"],
    "Ratings": [1,2,2,1,3,1,2]})

In [3]:
table

,city,TV,Ratings
0,Portland,KOIN,1
1,Portland,KABC,2
2,Sacramento,KCRA,2
3,Sacramento,KXTV,1
4,Austin,KVUE,3
5,Austin,KXAN,1
6,Austin,KTBC,2


In [4]:
table.pivot_table(columns = "city") # as table method

city,Austin,Portland,Sacramento
Ratings,2.0,1.5,1.5


What's the difference between ```pivot``` and ```pivot_table```?  The latter is introduced for its "aggregators" in the literature I've seen.  We reduce (summarize) along some axis (down or across) using one of many aggregator functions.  You may use more than one at once, by passing a list of same to ```aggfunc```.

By default (unless otherwise specified) the aggregator is the mean.

### LAB CHALLENGE:

How might you add some more rows to the above city table?  Feel free to add more for the same city, or add entirely new cities.  Avoid manually adding to the CSV file.  Think in terms of adding rows from within this Notebook, using any methods you have learned.

In [5]:
pd.pivot_table(table, columns = "city")  # invoked on module

city,Austin,Portland,Sacramento
Ratings,2.0,1.5,1.5


Does it really matter that we have these two forms?  What's the difference between convenience and cruft & clutter?  Core Python has some similar constructs so at least the duplication does not seem wholly alien.

### LAB CHALLENGE:

Modify the code cell above (or start with a copy) and override the default with an explicit aggregator function.  How about more than one aggregator function?

In [6]:
weather = pd.read_csv("weather.csv")

With thanks to [codebasics](https://github.com/codebasics/) which has a pandas teaching Youtube channel.  Check it out!

In [7]:
weather

,date,city,temperature,humidity
0,5/1/2017,new york,65,56
1,5/2/2017,new york,66,58
2,5/3/2017,new york,68,60
3,5/1/2017,mumbai,75,80
4,5/2/2017,mumbai,78,83
5,5/3/2017,mumbai,82,85
6,5/1/2017,beijing,80,26
7,5/2/2017,beijing,77,30
8,5/3/2017,beijing,79,35


The result of pivoting is usually a dataframe with a multi-index, which is the same thing as a hierarchical index.  The cities now need to repeat to give the two sets of temperatures.  Both tables contain 18 numerical cells.

### LAB CHALLENGE: 

What happens if you only have two rows of data for Mumbai?  Remove a row and find out.

In [8]:
weather.pivot(index="date", columns="city")

temperature                 humidity                
city         beijing mumbai new york  beijing mumbai new york
date                                                         
5/1/2017          80     75       65       26     80       56
5/2/2017          77     78       66       30     83       58
5/3/2017          79     82       68       35     85       60

In [9]:
weather.pivot(index="date", columns="city", values="humidity")

city,beijing,mumbai,new york
date,,,
5/1/2017,26,80,56
5/2/2017,30,83,58
5/3/2017,35,85,60


In [10]:
weather = pd.read_csv("weather2.csv")

In [11]:
weather

,date,city,temperature,humidity
0,5/1/2017,new york,65,56
1,5/1/2017,new york,61,54
2,5/2/2017,new york,70,60
3,5/2/2017,new york,72,62
4,5/1/2017,mumbai,75,80
5,5/1/2017,mumbai,78,83
6,5/2/2017,mumbai,82,85
7,5/2/2017,mumbai,80,26


In [12]:
weather.pivot_table(index="city", columns="date") # aggregates using mean by default

humidity          temperature         
date     5/1/2017 5/2/2017    5/1/2017 5/2/2017
city                                           
mumbai       81.5     55.5        76.5     81.0
new york     55.0     61.0        63.0     71.0

In [13]:
weather.pivot_table(index="city", columns="date", margins=True)

humidity                 temperature                 
date     5/1/2017 5/2/2017    All    5/1/2017 5/2/2017     All
city                                                          
mumbai      81.50    55.50  68.50       76.50     81.0  78.750
new york    55.00    61.00  58.00       63.00     71.0  67.000
All         68.25    58.25  63.25       69.75     76.0  72.875

### LAB CHALLENGE: 

Humidity only, with margins, please.

In [14]:
weather = pd.read_csv("weather3.csv")

In [15]:
weather["date"] = pd.to_datetime(weather["date"])

In [16]:
weather.pivot_table(index = pd.Grouper(freq ="M", key='date'), columns="city")

,humidity,temperature
city,new york,new york
date,,
2017-05-31,56.666667,65.333333
2017-12-31,51.000000,27.666667


In [17]:
weather.pivot_table(index = pd.Grouper(freq ="M", key='date'), columns="city")

,humidity,temperature
city,new york,new york
date,,
2017-05-31,56.666667,65.333333
2017-12-31,51.000000,27.666667


In [18]:
fcc = pd.read_msgpack("fcc_data.msg")  # one of the few fully round-trippable formats

In [19]:
fcc.loc["layer0":"layer1", :]

Vtype       ivm                xyz                    
Coords        a  b  c  d         x         y         z
Shell  Ball                                           
layer0 0      0  0  0  0  0.000000  0.000000  0.000000
layer1 1      0  1  1  2  0.000000 -0.707107 -0.707107
       2      0  1  2  1 -0.707107  0.000000 -0.707107
       3      0  2  1  1 -0.707107 -0.707107  0.000000
       4      1  0  1  2  0.707107  0.000000 -0.707107
       5      1  0  2  1  0.000000  0.707107 -0.707107
       6      1  1  0  2  0.707107 -0.707107  0.000000
       7      1  1  2  0 -0.707107  0.707107  0.000000
       8      1  2  0  1  0.000000 -0.707107  0.707107
       9      1  2  1  0 -0.707107  0.000000  0.707107
       10     2  0  1  1  0.707107  0.707107  0.000000
       11     2  1  0  1  0.707107  0.000000  0.707107
       12     2  1  1  0  0.000000  0.707107  0.707107

In [20]:
fcc.pivot_table(columns="Shell", aggfunc="count")

Shell         layer0  layer1  layer2  layer3  layer4
Vtype Coords                                        
ivm   a            1      12      42      92     162
      b            1      12      42      92     162
      c            1      12      42      92     162
      d            1      12      42      92     162
xyz   x            1      12      42      92     162
      y            1      12      42      92     162
      z            1      12      42      92     162

### LAB CHALLENGE: 

Might I collapse the above into just two rows, ```ivm``` and ```xyz```?